In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
import spacy
sp = spacy.load('en_core_web_sm')
from operator import itemgetter 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
# path to read the htmls
path = "/content/drive/My Drive/Wissee AI Projects/1. People/Zhengtong Pan/Lyst_helper/lyst_helper/data/"
# identify path to save the result
path_result = "/content/drive/My Drive/Wissee AI Projects/1. People/Zhengtong Pan/Lyst_helper/lyst_helper/results/"

### Parse Lyst Page

In [27]:
"""
parse lyst page
"""
from bs4 import BeautifulSoup
def clean_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    # soup = BeautifulSoup(html, "html.parser") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

# get the text of each page 
def get_page_text(path, file_name): 
    _in_html = path + file_name
    # print(_in_html)
    with open(_in_html) as _out:
        html = _out.read()
    text =  clean_html(html)
    return text

def get_text_from_all_pages(html_strings):
    # create a list of htmls
    # html_strings = "the-lyst-index-q119.html  the-lyst-index-q220.html  the-lyst-index-q320.html the-lyst-index-q120.html  the-lyst-index-q221.html  the-lyst-index-q418.html the-lyst-index-q121.html  the-lyst-index-q318.html  the-lyst-index-q419.html the-lyst-index-q219.html  the-lyst-index-q319.html  the-lyst-index-q420.html"
    html_lst_ = html_strings.split(" ")
    html_lst = [html for html in html_lst_ if html != ""]

    # get text from all pages
    all_pages_text = []
    for html in html_lst: 
        page_text = get_page_text(path, html)
        all_pages_text.append(page_text)
    return all_pages_text

###Extract Notes
- split text into list of lists
- regex to find the sublist that contains percentage
- if  the sentence contaisn "%" and starts with verb, combine the sentence with its previous sentence, and the sentence after it. 

In [6]:
def get_string_with_percenatge(text):
    text_lst = text.split("\n")
    # print(text_lst)
    percenatge_string_lst = []
    indexes = []
    for i in range(len(text_lst)): 
        string = text_lst[i]
        str_tok = word_tokenize(string)
        pos_tag = nltk.pos_tag(str_tok)
        # print(str_tok)
        # print(nltk.pos_tag(str_tok))
        # check whether the string contains number with percenatage or not 
        m = re.match(r'\d+%|([0-9]\d?)\.\d+%', string)
        # print(m.string)
        if "%" in string:
          # percenatge_string_lst.append(m.string)
          # string with percentage but starts with verb or start with number with percenatge only 
          if (pos_tag[0][1] == "VBD" or m ) and i!=0 and i!=len(text_lst)-1:
              tmp_before = text_lst[i-1]
              tmp_after = text_lst[i+1]
              res = tmp_before + " " + string
              res = res + " " + tmp_after
              percenatge_string_lst.append(res)
              indexes.append(i)
          elif re.match(r'NN', pos_tag[0][1]) or re.match(r"JJ", pos_tag[0][1]): # sentence with percentage starts with noun or descriptive noun
              percenatge_string_lst.append(string)
              indexes.append(i)
    return percenatge_string_lst, indexes

# input: all pages text (list of lists; sublist is a html page)
# output: all strings with percentage (list of lists; sublist is the filtered string from each html page)
def get_all_strings_with_percentage(all_pages_text):
    all_strings_with_percentage = []
    all_indexes = []
    for page_text in all_pages_text: 
        percenatge_string_lst, indexes= get_string_with_percenatge(page_text)
        all_strings_with_percentage.append(percenatge_string_lst)
        all_indexes.append(indexes)
    return all_strings_with_percentage, all_indexes

### Extract Quarter Year

In [8]:
# input: page text
# output: all the quarter year information, sentences with percentage
def get_string_with_quarter_year(text):
    text_lst = text.split("\n")
    # print(text_lst)
    quarter_year_lst = []
    percentage_string_lst =[]
    quarter_year_indexes = []
    percentage_indexes = []
    for i in range(len(text_lst)): 
        string = text_lst[i]
        if re.match(r"Q", string):
            quarter_year_lst.append(string)
            quarter_year_indexes.append(i)
        if "%" in string:
            percentage_string_lst.append(string)
            percentage_indexes.append(i)
    return quarter_year_lst, percentage_string_lst, quarter_year_indexes, percentage_indexes

def get_all_strings_with_quarter_year(all_pages_text):
    all_strings_quarter_year = []
    all_strings_with_percenatges = []
    all_quarter_year_ind = []
    all_string_with_percentage_ind = []
    for page_text in all_pages_text:
        quarter_year_lst, percentage_string_lst, quarter_year_indexes, percentage_indexes = get_string_with_quarter_year(page_text)
        all_strings_quarter_year.append(quarter_year_lst)
        all_strings_with_percenatges.append(percentage_string_lst)
        all_quarter_year_ind.append(quarter_year_indexes)
        all_string_with_percentage_ind.append(percentage_indexes)
    return all_strings_quarter_year, all_strings_with_percenatges, all_quarter_year_ind, all_string_with_percentage_ind

### Extract Brands

In [10]:
def identify_entity_text(text, ent_label): 
    doc = sp(text)
    entity_text = []
    if doc.ents:
       for ent in doc.ents:
           if ent.label_ == ent_label:
              entity_text.append(ent.text)
          # if ent.label_ == "DATE":
              # years.append(ent.text)
    if (entity_text  == []) and ent_label == "ORG":
        text_tok = word_tokenize(text)
        pos_tags = nltk.pos_tag(text_tok)
        # print(pos_tags)
        for i in range(len(pos_tags)):
            # identify. the text that contains number with percenatge
            if i!= len(pos_tags)-1 and (pos_tags[i][1] == "CD" and pos_tags[i+1][0] == "%"):  
              candidates = pos_tags[:i] # candidates to find product/brand names
              # find indexes of noun
              product_brand_indexes = [i for i in range(len(candidates)) if candidates[i][1] in("NN", "NNP")]
              # print(product_brand_indexes)
              # get product/brands
              product_brand_tags = itemgetter(*product_brand_indexes)(candidates)
              if type(product_brand_tags) == tuple and len(product_brand_tags) == 2: # corner case for a single tuple
                  # print(product_brand_tags[0])
                  product_brands = [product_brand_tags[0]]
                  # print(product_brands)
              else:
                  product_brands = [tag[0] for tag in product_brand_tags]
                  entity_text.append(product_brands)
                  # print(entity_text)
                  break
    elif entity_text == []:
          entity_text.append(None)
    return entity_text

def get_all_entity_text(notes, ent_label, all_notes_indexes):
    all_brands = []
    all_brands_ind = []
    for i in range(len(notes)): # for each page 
        page_brands = []
        page_brand_ind = []
        for s in range(len(notes[i])): # for each sentence in the page 
            brands= identify_entity_text(notes[i][s], ent_label)
            page_brands.append(brands)
            if brands != [None]: 
               page_brand_ind.append(all_notes_indexes[i][s])
        all_brands.append(page_brands)
        all_brands_ind.append(page_brand_ind)
    return all_brands, all_brands_ind

### Data Cleaning

#### Year Cleaning:
- Match the year with notes: 
- Get the indexes of the year
- Get the indexes of the strings with percentage 
- Assign the year with the index closest to the index of the string with percentage

##### Algorithm to find the closest year: 

for each string_index in Percentage_String_Index:
 - locate the index within the Year_Index
 - find the largest year_index which is smaller than the string_Index
 - page of year for the string_index is the largest year_index smaller than the string_Index

In [15]:
# binary search to find the largest element that is smller or equal to the current index
def find_largest_smaller(array, target):
    if not array or len(array) == 0: 
       return array[-1]
    left = 0
    right = len(array)-1
    while left < right -1: 
        mid = (left + right)//2
        # case1: mid point value < target 
        if array[mid] < target: 
           left = mid
        # case2: mid point value > target
        elif array[mid] > target: 
            right = mid-1
        # case3: mid point value == target
        else: 
            left = mid
    # post-processing
    # see wehther right or left <= target or neither of them <= target 
    if array[right] <= target: 
       return array[right]
    if array[left] <= target:
       return array[left]
    else: 
       return array[-1]

# create percentage string index and percentage string dictionary 
# year index and year/quarter value dictionary 
def create_dictionary(value_lst, index_lst):
    dict_from_list = dict(zip(index_lst, value_lst))
    return dict_from_list

def create_feature_dictionary(df_tmp, i):
    notes_dict = create_dictionary(df_tmp['Notes'][i], df_tmp['Notes_indexes'][i])
    year_dict = create_dictionary(df_tmp['Quarter/Year'][i], df_tmp['Year_Index'][i])
    brand_dict = create_dictionary(df_tmp['Product/brand'][i], df_tmp['Product/brand_indexes'][i])

    # add percentage string into first_page_notes_dict if it doesn't have the percenatage string 
    extra_notes_indexes = list(set(df_tmp['Percentage_String_Index'][i]) - set(df_tmp['Notes_indexes'][i]))
    extra_notes = list(set(df_tmp['Percentage_strings_only'][i]) - set(df_tmp['Notes'][i]))
    extra_notes_dict = create_dictionary(extra_notes, extra_notes_indexes)
    notes_dict.update(extra_notes_dict)
    return notes_dict, year_dict, brand_dict

# get the year index (largest number smaller than current percentage string index) for each percentage string 
# get the year value based on year index for each percentage string 
def create_unwrap_dataframe(df_tmp, notes_dict, year_dict, brand_dict, page_ind):
    df_example = pd.DataFrame(columns = ['Quarter/Year', "Product/brand", 'Notes'])
    i = 0
    for str_ind in notes_dict.keys(): 
        year_ind = find_largest_smaller(df_tmp['Year_Index'][page_ind], str_ind)
        quarter_year = year_dict[year_ind]
        percentage_string = notes_dict[str_ind]
        if str_ind in brand_dict:
            brand = brand_dict[str_ind]
        else: 
            brand = None
        df_example.loc[i] = [quarter_year] + [brand] + [percentage_string]
        i += 1
    return df_example

def create_all_dataframes(df_raw):
    frames = None
    for page_ind in range(df_raw.shape[0]): 
        notes_dict, year_dict, brand_dict = create_feature_dictionary(df_raw, page_ind)
        df_curr = create_unwrap_dataframe(df_raw, notes_dict, year_dict, brand_dict, page_ind)
        if frames is None: 
           frames = df_curr
        else:
           result =  pd.concat([frames, df_curr])
           frames = result
    return frames

#### Brands Cleaning
- Get the organization entity closest to each percentage string within a setence to assign brands
- Get the indexes of extracted brands and extracted percentage strings
- Use binary search(find largest smaller) within each sentence to find the closest brand for each sentence

In [20]:
def get_extra_brands(df_null_brands):
    all_extra_brands = []
    all_extra_brands_ind = []
    all_percent_ind = []
    for note in df_null_brands['Notes']: 
        # create entity for each note
        extra_brands = []
        extra_brands_ind = []
        percent_ind = []
        doc = sp(note)
        if doc.ents: 
          for i in range(len(doc.ents)):
              # recognize percentage and record its index 
              if doc.ents[i].label_ == "PERCENT":
                  percent_ind.append(i)
              if doc.ents[i].label_ == "ORG":
                  extra_brands.append(doc.ents[i].text)
                  extra_brands_ind.append(i)
          all_extra_brands.append(extra_brands)
          all_extra_brands_ind.append(extra_brands_ind)
          all_percent_ind.append(percent_ind)
        else:
          all_extra_brands.append(None)
          all_extra_brands_ind.append(None)
          all_percent_ind.append(None)
    return all_extra_brands, all_extra_brands_ind, all_percent_ind

def get_closest_brands(df_extra_brands):
    # assumptions: percentage index is not null; brands can be; number of percent indexes < number of brand indexes 
    clean_extra_brands = []
    for i in range(df_extra_brands.shape[0]):
        percent_lst = df_extra_brands['All_Percent_Ind'][i]
        brand_ind_lst = df_extra_brands['Extra_Brands_Ind'][i]
        extra_brands_lst = df_extra_brands['Extra_Brands'][i]
        # case1: same amount of percentages and brands 
        if len(percent_lst) == len(brand_ind_lst):
          closest_brands = extra_brands_lst
          # print(closest_brands)
        # case2: 1 index in the percent_lst and >1 indexes in Extra_Brands_Ind
        elif len(percent_lst) ==1 and len(brand_ind_lst)>1:
          # print(percent_lst[0])
          # print(brand_ind_lst)
          # binary searh to find largest smaller elements(index)
          closest_brand_ind = find_largest_smaller(brand_ind_lst, percent_lst[0])
          # print(closest_brand_ind)
          # get actual brand index within brand names list 
          actual_brand_ind = brand_ind_lst.index(closest_brand_ind)
          # print(actual_brand_ind)
          # get the closest brand
          closest_brands = [extra_brands_lst[actual_brand_ind]] 
          # print(closest_brands)
        # case3: >1 indexes in the percent_st and >1 indexes in EXtra_Brands_Ind
        elif len(percent_lst) > 1 and len(brand_ind_lst) > 1: 
            # for each percent_ind in percent_lst, find the closest brand_ind to it 
            closest_brands = []
            for percent_ind in percent_lst: 
                # get closest brand index 
                tmp_ind = find_largest_smaller(brand_ind_lst, percent_ind)
                # get actual brand indexes within brand names list 
                actual_brand_ind = brand_ind_lst.index(tmp_ind)
                closest_brand = [extra_brands_lst[actual_brand_ind]]
                closest_brands.append(closest_brand)
            # print(closest_brands)
        # case4: can't find brand within the percenatge sentence
        else: 
            closest_brands = [None]
        # print(closest_brands)
        clean_extra_brands.append(closest_brands)
    return clean_extra_brands

def unpack_list(lst):
    return [item for sublist in lst for item in sublist]

### Execute all steps in a flow

In [35]:
def main():
    # get text from all pages
    html_strings = input("Enter htmls formatted with qoutes and seperated by a single white space:")
    all_pages_text = get_text_from_all_pages(html_strings)
    # extract notes
    # get all strings with percentages 
    all_strings_with_percentage, all_notes_indexes = get_all_strings_with_percentage(all_pages_text)
    notes = pd.Series(all_strings_with_percentage)
    # extract quarter year
    all_quarter_years, all_strings_percentages_only, all_quarter_year_ind, all_string_with_percentage_ind= get_all_strings_with_quarter_year(all_pages_text)
    # extract brands
    all_brands, all_brands_ind = get_all_entity_text(notes, "ORG", all_notes_indexes)

    # data cleaning 

    # year cleaning
    # non-cleaned dataframe
    # dataframe with year, percentage string indexes 
    frame = {'Quarter/Year': all_quarter_years, "Year_Index": all_quarter_year_ind, "Product/brand": all_brands, "Product/brand_indexes": all_brands_ind,  "Notes": notes, "Notes_indexes": all_notes_indexes, "Percentage_strings_only": all_strings_percentages_only, "Percentage_String_Index": all_string_with_percentage_ind}
    df = pd.DataFrame(frame)
    df_tmp = df[['Quarter/Year',	'Year_Index', "Product/brand", "Product/brand_indexes", 'Percentage_strings_only',	'Percentage_String_Index', "Notes", "Notes_indexes"]]
    df_all = create_all_dataframes(df_tmp)
    # further cleaning of year data
    new_quarter_year = []
    for year in df_all['Quarter/Year']:
        if "HOTTEST WOMEN'S PRODUCTS" in year: 
            new_year = year.replace("HOTTEST WOMEN'S PRODUCTS", "")
            new_quarter_year.append(new_year)
        else:
            new_quarter_year.append(year)
    df_all['Quarter/Year'] = new_quarter_year
    df_all[['Quarter', "Year"]] = df_all['Quarter/Year'].str.split(expand=True)
    df_all.drop(['Quarter/Year'], axis=1, inplace=True)
    # reorganize the columns
    df_all_new = df_all[["Year", "Quarter", "Product/brand", "Notes"]]

    #brands cleaning
    all_filter_brands, all_filter_brands_ind, all_percent_ind = get_extra_brands(df_all_new)
    filter_brands_frame = {"Extra_Brands": all_filter_brands, "Extra_Brands_Ind": all_filter_brands_ind, "All_Percent_Ind": all_percent_ind}
    df_filter_brands = pd.DataFrame(filter_brands_frame)

    # format all closest brands into one list of brands per sentence
    all_closest_brands = get_closest_brands(df_filter_brands)
    all_closest_brands_unpack = [ unpack_list(brands)if len(brands)>1 and type(brands[0]) == list else brands for brands in all_closest_brands]
    # replace cleaned brand values for all brands  
    df_filter_brands = df_all_new.copy()
    df_filter_brands['Product/brand'] = all_closest_brands_unpack

    # save the result
    df_filter_brands.to_excel(path_result + "lyst_fashion_trend_final.xlsx")

In [36]:
main()

Enter htmls formatted with qoutes and seperated by a single white space:the-lyst-index-q119.html  the-lyst-index-q220.html  the-lyst-index-q320.html the-lyst-index-q120.html  the-lyst-index-q221.html  the-lyst-index-q418.html the-lyst-index-q121.html  the-lyst-index-q318.html  the-lyst-index-q419.html the-lyst-index-q219.html  the-lyst-index-q319.html  the-lyst-index-q420.html
